In [ ]:
from __future__ import print_function
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
import io
import requests

In [ ]:
def getSparkSessionInstance(sparkConf):
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession\
            .builder\
            .config(conf=sparkConf)\
            .getOrCreate()
    return globals()['sparkSessionSingletonInstance']


if __name__ == "__main__":
    sc = SparkContext.getOrCreate()
    ssc = StreamingContext(sc, 5)

In [ ]:
socket_stream = ssc.socketTextStream("192.1.1.1", 5559)

In [ ]:
lines = socket_stream.window(60)

In [ ]:
tweets = lines.flatMap(lambda line: line.split("\n"))

In [ ]:
# Convert RDDs of the words DStream to DataFrame and run SQL query
def process(time, rdd):
        print("========= %s =========" % str(time))
        from pyspark.sql.types import NumericType
        try:
            # Get the singleton instance of SparkSession
            spark = getSparkSessionInstance(rdd.context.getConf())

            header = ["ScreenName","FollowersCount","Language","TweetsCount","CountryCode"]
            rowRdd = rdd.map(lambda row : row.split("||"))
            jsonDataFrame = spark.createDataFrame(rowRdd, header)
            jsonDataFrame = jsonDataFrame.withColumn("FollowersCount",jsonDataFrame["FollowersCount"].cast("Int"))
            jsonDataFrame = jsonDataFrame.withColumn("TweetsCount",jsonDataFrame["TweetsCount"].cast("Int"))
            
                        
            # Creates a temporary view using the DataFrame.
            jsonDataFrame.createOrReplaceTempView("tweets")
            #5. Find Top tweeting user:
            #5.1 Find User who is tweeting a lot.
            #5.2 Find top 50 across the world.

            TweetCountDF = \
            spark.sql("select ScreenName,TweetsCount  from tweets order by TweetsCount desc limit 20")
            TweetCountDF.show()
            send_df_to_dashboard(TweetCountDF)
            
        except:
            pass

        

In [ ]:
def send_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.ScreenName) for t in df.select("ScreenName").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.TweetsCount for p in df.select("TweetsCount").collect()]
    # initialize and send the data through REST API
    url = 'http://192.1.1.1:5001/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [ ]:
tweets.foreachRDD(process)

In [ ]:
ssc.start()

In [ ]:
ssc.awaitTermination()